# Import Python Module

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from haversine import haversine

In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
# pyspark.sql.maxToStringFields=100

In [5]:
from pyspark import SparkContext

[설정](https://steemit.com/cloudera/@uneedcomms/dw-ver-2-spark)  
을 분명 아래처럼 하는데 뭔가  
그 Executor에 바뀐게 하나도 없음(논란거리)  
도대체 저번에 11GB 바뀐게 뭔지가 의문

## +++  
막 이런 spark tuning 예시들 보면 다들 스펙 장난아니더라  
cpu 16코어 메모리 120GB 짜리 4 server로 spark 돌려서(대신 aws인듯)   
기본 튜닝하면 되게 크게 설정이 가능한데  
소올직히 로컬(나는 찾아보니 cpu 4코어 메모리 16GB)에서는 튜닝해도 별 감흥 없음...  

In [6]:
SparkContext.setSystemProperty('yarn.nodemanager.resource.memory-mb', '14g')

In [7]:
SparkContext.setSystemProperty('yarn.nodemanager.resource.cpu-vcores', '3')

In [8]:
SparkContext.setSystemProperty('yarn.scheduler.maximum-allocation-mb', '14g')

In [9]:
SparkContext.setSystemProperty('yarn.scheduler.minimum-allocation-mb', '1g')

In [10]:
SparkContext.setSystemProperty('spark.executor.cores', '3')

In [11]:
SparkContext.setSystemProperty('spark.executor.instances', '1')

In [12]:
SparkContext.setSystemProperty('spark.executor.memory', '3g')

In [13]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import col

In [14]:
sc = SparkContext("local", "App Name")
sql2 = SQLContext(sc)

In [15]:
spark = SparkSession.builder.getOrCreate()

# =============================================

In [23]:
seq = range(1,100)

In [24]:
spark_da = sc.parallelize(seq)

In [25]:
spark_da

PythonRDD[1] at RDD at PythonRDD.scala:52

# Open Crime Data File

In [27]:
data = spark.read.csv("./Crimes_Chicago.csv", header=True)

In [28]:
type(data)

pyspark.sql.dataframe.DataFrame

[RDD로 csv 파일 열기](https://stackoverflow.com/questions/43239969/read-csv-file-with-strings-to-rdd-spark)  
[RDD 관련 정보1](https://bcho.tistory.com/1027?category=563141)  

In [29]:
rdd_data = spark.read.csv("./Crimes_Chicago.csv", header=True).rdd

In [30]:
type(rdd_data)

pyspark.rdd.RDD

In [32]:
rdd_data.getNumPartitions()

14

## ------------------------------------------------------------------------------

In [39]:
def DropNaindata(data):
    return len(data) != 0

In [40]:
usedef = rdd_data.filter(DropNaindata)

### 오 rddfile.count() 하면 count 나와야하는데 엄청 오래 걸림...^^;  
rdd도 답이 아닌가..  
Google cloud 걔 써야하나 싶고 그렇다....

## ------------------------------------------------------------------------------

In [ ]:
chicago = data.alias("chicago")

In [ ]:
chicago = data.na.drop(subset =["Longitude"])

In [ ]:
chicago.createOrReplaceTempView("CHICAGO")
year_count = spark.sql("SELECT Year, count(Year) AS Count FROM CHICAGO GROUP BY Year ORDER BY Year")

In [ ]:
# 복사
chicago_md = chicago

In [ ]:
chicago_md = chicago_md.withColumn('Month', chicago_md['Date'].substr(1,2))
chicago_md = chicago_md.withColumn('Day', chicago_md['Date'].substr(4,2))
chicago_md = chicago_md.withColumn('Time', chicago_md['Date'].substr(12,15))

* Rename columns

In [ ]:
chicago_md_rename = chicago_md.select('ID',col("Case Number").alias("Case_Number"), 'Date','Block','IUCR',
                                      col("Primary Type").alias("Primary_Type"),'Description',
                                      col("Location Description").alias("Location_Description"), 
                                      'Arrest','Domestic','Beat','District','Ward',
                                      col("Community Area").alias("Community_Area"),
                                      col("FBI Code").alias("FBI_Code"),
                                     col("X Coordinate").alias("X_Coordinate"),col("Y Coordinate").alias("Y_Coordinate"),
                                      col("Updated On").alias("Updated_On"),'Latitude','Longitude','Location',
                                     col("Historical Wards 2003-2015").alias("Historical_Wards_20032015"),
                                     col("Zip Codes").alias("Zip_Codes"),col("Community Areas").alias("Community_Areas"),
                                     col("Census Tracts").alias("Census_Tracts"), 'Wards',
                                      col("Boundaries - ZIP Codes").alias("Boundaries_ZIPCodes"),
                                     col("Police Districts").alias("Police_Districts"), col("Police Beats").alias("Police_Beats"),
                                      'Year','Month', 'Day','Time')

In [ ]:
chicago_md_rename.createOrReplaceTempView("CHICAGO_MD_RE")

* 년도별 분류

In [ ]:
year_lst = list(year_count.select('Year').toPandas()['Year'])

In [ ]:
yearlst = []
for i in range(len(year_lst)):
#     print(year_lst[i])
    tmp = spark.sql("SELECT * FROM CHICAGO_MD_RE WHERE Year = {}".format(year_lst[i]))
    yearlst.append(tmp)

In [ ]:
crime1617 = yearlst[15].union(yearlst[16])
crime1718 = yearlst[16].union(yearlst[17])
crime1819 = yearlst[17].union(yearlst[18])

# Open School Data

In [ ]:
school = spark.read.csv("./school_profile.csv", header=True)

In [ ]:
school_loc = school.select("School_ID","School_Latitude","School_Longitude","SY")

## * School Income

In [ ]:
school_income = school.select('School_ID','Student_Count_Total','Student_Count_Low_Income')

In [ ]:
school_income = school_income.withColumn("Low_Income_Ratio", 
                         (col("Student_Count_Low_Income")/col("Student_Count_Total")))                        

In [ ]:
# school_income.sort(col("Low_Income_Ratio").desc()).show()

In [ ]:
school_income_ratio = school_income.select("School_ID","Low_Income_Ratio").sort(col("Low_Income_Ratio").desc())

## School time join School loc

In [ ]:
# key 중복 없이 JOIN 하는 방법

join_school_loc_ratio = school_loc.join(school_income_ratio, ["School_ID"], how = 'left')

* SY 기준 분류

In [ ]:
t_sy_lst = join_school_loc_ratio.select("SY").distinct().toPandas()["SY"].tolist()

In [ ]:
sc_yearlst_time = []
for i in range(len(t_sy_lst)):
#     print(t_sy_lst[i])
    tmp = join_school_loc_ratio.select(col("*")).where(col("SY") == t_sy_lst[i])
    sc_yearlst_time.append(tmp)

In [ ]:
t_sy_lst

# Crime Location & School Location

* EX) 1617

In [ ]:
crime1617_df = crime1617.select('Primary_Type', 'Latitude', 'Longitude')

In [ ]:
cross_join = sc_yearlst_time[0].crossJoin(crime1617_df)

In [ ]:
cross_join.createOrReplaceTempView("DISTANCE")

In [ ]:
distance_table = spark.sql("SELECT *, ROUND((6371*acos(cos(radians(School_Latitude))*cos(radians(Latitude))*cos(radians(Longitude)-radians(School_Longitude))+sin(radians(School_Latitude))*sin(radians(Latitude)))),6) AS Distance FROM DISTANCE")

* CSV 저장

In [ ]:
distance_group1_1 = distance_table.filter(distance_table.Distance <=2).groupby("School_ID").count()

In [ ]:
distance_group1_1.show()

# ===========================================